# Analiza podatkov za projektno nalogo

Pozdravljeni,
v tem zvežčiču bom analiziral uroke, katere sem pobral iz spletne strani [dnd5e][1]. Uroki so sicer plačljivo gradivo, ki je raztrošeno po mnogih knjigah lastnika igre *Dungeons and Dragons*, ampak ker so hkrati dostopni tudi na tej brezplačni spletni strani, bom rekel, da je moja analiza nesporna. 

## Nekaj malega o igri

Najpomembnejši v igri so igralci z njihovimi junaki. Junak je oseba, kateri igralec določi raso, ceh, ozadje in tekom igre še veliko več. Kar se tiče izdelave junaka je igralcem dana precejšna svoboda, ampak za lažji potek in uravnovešenost igre se je pametno držati v naprej podanih okvirjev. Tako junaki pridobijo kompetence za spopadanje z nalogami, ki jim jih zadajo [NPC][2]-ji in zmožnost bojevanja. Tu imajo uroki pomembno vlogo, saj jih igralec lahko uporablja bodisi na vsakodnevni ravni za razreševanje problemov bodisi v boju za okrevanje in zadajanje škode. 

## Lastnosti urokov

Vsak urok ima osnovne karakteristike, ki so: 

- ime (`uroki.csv`), 
- šolo čaranja (`sola.csv`), 
- čas potreben za izvedbo uroka (`urocanje.csv`), 
- možnost izvedbe uroka v obliki rituala (`uroki.csv`),
- doseg (`doseg.csv`), 
- čas, ko je urok aktiven (`efekt.csv`),
- in za izvedbo potrebne komponente (materialne, somatske in verbalne) (`uroki.csv`),
- stopnjo uroka, ki je med 0 in 9 (`uroki.csv`).

Te podatke sem pobral s svetovnega spleta in jih uredil v `csv` datoteke. Poleg teh lastnosti imajo uroki tudi natančne opise, do katerih lahko dostopate preko `http://dnd5e.wikidot.com/spell:{urok_href}`, kjer je za vsak urok `urok_href` možno najti v datoteki `uroki.csv`.



[1]: <http://dnd5e.wikidot.com/spells> "Uroki v D&D"
[2]: <https://en.wikipedia.org/wiki/Non-player_character> "Več o NPC-jih"

In [2]:
import pandas as pd

In [3]:
uroki = pd.read_csv('podatki/uroki.csv')

In [4]:
uroki

,urok_href,urok,sola,urocanje,doseg,efekt,ritual,verbal,somatic,material,stopnja
0,acid-splash,Acid Splash,1,1,1,1,False,True,True,False,0
1,blade-ward,Blade Ward,2,1,2,2,False,True,True,False,0
2,booming-blade,Booming Blade,3,1,3,2,False,False,True,True,0
3,chill-touch,Chill Touch,4,1,4,2,False,True,True,False,0
4,control-flames,Control Flames,5,1,1,3,False,False,True,False,0
...,...,...,...,...,...,...,...,...,...,...,...
556,time-stop,Time Stop,5,1,2,1,False,True,False,False,9
557,true-polymorph,True Polymorph,5,1,11,39,False,True,True,True,9
558,true-resurrection,True Resurrection,4,5,5,1,False,True,True,True,9
559,weird,Weird,8,1,9,4,False,True,True,False,9


In [5]:
sola = pd.read_csv('podatki/sola.csv')

In [6]:
urocanje = pd.read_csv('podatki/urocanje.csv')

In [7]:
doseg = pd.read_csv('podatki/doseg.csv')

In [8]:
efekt = pd.read_csv('podatki/efekt.csv')

## Urediev podatkov

Preden začnem z analizo podatkov, moram prečistiti štiri pomožne datoteke, ker program ki jih je uredil ni opazil razlik med vsebinsko istimi a drugače zapisanimi vrednostmi.

To je vidno v datoteki `urocanje.csv`, kjer se isti časi razlikujejo le za veliko začetnico.

In [9]:
urocanje

,ime,id
0,1 Action,1
1,1 Bonus Action,2
2,1 Minute,3
3,1 Reaction,4
4,1 Hour,5
5,10 Minutes,6
6,Reaction,7
7,1 action,8
8,Self,9
9,1 hour,10
